Article collection from http://www.geologica-acta.com. They aggraid with data collection.

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd
import random
import os

In [2]:
version = "1.0.0"
datadir = "data/"
archivedir = "data/Archive"

# Get the Article list

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(filename):
    # open old file
    olddf = pd.DataFrame({'url': [] })
    try:
        olddf = pd.read_pickle(os.path.join(archivedir, filename))
        print(olddf.tail())
    except:
        pass
    
    driver.get("https://revistes.ub.edu/index.php/GEOACTA/search/titles")
    
    # itterate over the journal searching to get the Article list
    articles = []
    def pagefind():
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # found the Article
        for tr in soup.find_all("tr"):
            # if the row is the ones which have the information
            if tr.has_attr("valign"):
                if tr['valign'] == "top":
                    # get the 3td whch is the link
                    td = tr.find_all("td")[2]
                    # here the first on is the link to the article
                    for a in td.find_all('a'):
                        articles.append(a["href"])
                        break
        # find next
        # iterate over links and find ">"
        for a in driver.find_elements_by_tag_name('a'): 
            if a.text.strip() == ">":
                try:
                    a.click()
                    sleep(2)
                    return True
                except Exception as e:
                    print("Error to go next page", str(e))
                    return False
        return False    

    nextpage = True
    while  nextpage:
        nextpage = pagefind()                
        print(driver.current_url, end='\r')
        
    # filer out old articles
    oldarticles = olddf['url']
    articles = list( set(articles).difference(set(oldarticles) ))
    print(len(articles), "new article found!")
    
    # make df
    df = pd.DataFrame({'url': list(set(articles)), 
                       'journal_title': "Geologica Acta", 
                       'journal_eissn': "1696-5728",
                       'journal_pissn': '1695-6133',
                       'category': "Geology"
                      })
    # chek there was a not Arhived but previously loaded file
    adf = None
    try:
        adf = pd.read_pickle(os.path.join(datadir, filename))
    except:
        pass
    if adf is not None:
        df = pd.concat([df, adf])
        df.drop_duplicates(inplace=True)
    
    return df, olddf   

# Process new Article

In [5]:
def getarticledetails(df, olddf, filename):
    
    articles = df['url'].values

    titles = [ None for _ in range(len(df))]
    abstracts = [ None for _ in range(len(df))]
    writers = [ None for _ in range(len(df))]
    dates = [ None for _ in range(len(df))]
    dois = [  None for _ in range(len(df)) ]
    keywords = [  None for _ in range(len(df)) ]

    for idx in range(len(articles)):
        sleep(3)
        
        # print percentiage of the process
        print( str(np.round(100*idx/len(df),2))+"%" , end='\r')
    
        url = df.iloc[idx]['url']
        print(url)
        request = urllib.request.Request(url)
        request.add_header('Accept-Encoding', 'utf-8')
        try:
            response = urlopen(request)    
            page_content = response.read().decode('utf-8')
        except:
            continue
    
        page_soup = BeautifulSoup(page_content, features="lxml")

        # keywords, writers and dates
        writter = []
        for m in page_soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == 'DC.Type.articleType':
                    if  m['content'].strip() != 'Articles':
                        print("Not a Article")
                        continue
                if m['name'] == "keywords":
                    keywords[idx] =  m['content'].replace(";", "#")
                if m['name'] == "DC.Date.issued":
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y-%m-%d").strftime('%Y-%m-%d')
                if m['name'] == 'DC.Creator.PersonalName':
                    writter.append(m['content'])
                if m['content'] == 'DC.Identifier.DOI' or m['content'] == 'citation_doi':
                    doi[idx] = m['content']                    
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        
        # abstract
        abstract = ""
        try:
            abstdiv = page_soup.find("div", {"id": "articleAbstract"}) 
            # find the firs div as there is the text
            abstract = abstdiv.find('div').get_text().replace("\xa0", " ")              
        except Exception as e:
            print("No abstract for ", url, str(e))
            continue
        if len(abstract) > 0:
            abstracts[idx] = abstract
        else:
            print("No abstract for ", url)
            # if no abstract we ignore the page
            continue
        
        # title
        titles[idx] = page_soup.find('title').get_text().split("|")[0] 
        # if all title is capital we change it just the first letter
        fullcapital = True
        titletext = re.sub(r'[^a-zA-Z]', '', titles[idx], flags=re.UNICODE)
        for l in titletext :
            if l.isupper() is False:
                fullcapital = False
                break
        if fullcapital:
            titles[idx] = titles[idx].lower().title()                    
        
                    
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # merge owith old df
    df = pd.concat([df, olddf])
    
    # save data
    df.to_pickle(os.path.join(datadir, filename))
    
    # test
    test = pd.read_pickle(os.path.join(datadir, filename))
    print(test.head())                

# Main

In [ ]:
filename = 'journal_Geologica Acta_'+version+'.pandas'
    
# search for articles
df, olddf = gedjournalarticles( filename )
    
# get the articles details
getarticledetails(df, olddf, filename)